In [1]:
# Importing the required libraries
from IPython.display import display
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch.autograd import Variable
from torch_geometric.nn import GCNConv
from torch.utils.data import TensorDataset, DataLoader
import os
import glob
from sklearn.preprocessing import MinMaxScaler


def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

In [2]:
class Net(torch.nn.Module):
    def __init__(self, d_feat, edge_indexs, nnode):
        super(Net, self).__init__()
        self.edge_index = edge_indexs
        self.nnode = nnode
        self.conv1 = GCNConv(d_feat, 32, cached=True,
                             normalize= True)
        self.conv2 = GCNConv(32, 16, cached=True,
                             normalize=True)
        self.conv3 = GCNConv(16, 8, cached=True,
                             normalize=True)
        self.linears = torch.nn.ModuleList([torch.nn.Linear(8, 1) for _ in range(nnode)])


    def forward(self, x):
        x, edge_index, edge_weight = x, self.edge_index, None
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        
        out = [self.linears[i](x[i, :]) for i in range(self.nnode)]

        return out

## Processing the file to input into the model

In [3]:
# Specifying the path of file
path= r"F:\GAMES Research\GAT\GAT\DOE_Q1\L1\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['MIDATL.csv', 'SOUTH.csv', 'WEST.csv']


In [4]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[0]))

,Net,Temperature
0,31660.636,-7.6
1,30766.066,-8.4
2,30201.596,-8.3
3,29901.406,-6.8
4,30084.427,-7.0
...,...,...
43819,31844.076,4.8
43820,30687.078,4.4
43821,29583.295,3.7
43822,28458.691,2.8


## Processing the load data into train-test split

In [5]:
# Fetching and concatenating the data
Load_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Net"]) for fileName in fileList], axis= 1)
Load_DS.columns = [i.removesuffix('.csv') for i in fileList]
Load_DS = Load_DS.add_prefix("LOAD_")

Load_DS

,LOAD_MIDATL,LOAD_SOUTH,LOAD_WEST
0,31660.636,12571.309,49769.768
1,30766.066,12449.182,48714.914
2,30201.596,12295.476,47880.328
3,29901.406,12354.318,47380.142
4,30084.427,12542.606,47241.559
...,...,...,...
43819,31844.076,11682.100,49258.613
43820,30687.078,11442.870,47886.874
43821,29583.295,11166.770,46582.286
43822,28458.691,10838.550,45132.982


In [6]:
# Creating 1-day lag loads
Load_lag_1 = Load_DS.shift(24).fillna(0)
Load_lag_1.columns = [i.removeprefix('LOAD_') for i in Load_lag_1.columns]
Load_lag_1 = Load_lag_1.add_prefix("LAG1_")

Load_lag_1

,LAG1_MIDATL,LAG1_SOUTH,LAG1_WEST
0,0.000,0.000,0.000
1,0.000,0.000,0.000
2,0.000,0.000,0.000
3,0.000,0.000,0.000
4,0.000,0.000,0.000
...,...,...,...
43819,33159.726,10860.510,49037.954
43820,32353.072,10608.808,48438.677
43821,31132.848,10213.428,47229.754
43822,29356.335,9765.158,45404.894


In [7]:
# Creating 7-day lag loads
Load_lag_7 = Load_DS.shift(24*7).fillna(0)
Load_lag_7.columns = [i.removeprefix('LOAD_') for i in Load_lag_7.columns]
Load_lag_7 = Load_lag_7.add_prefix("LAG7_")

Load_lag_7

,LAG7_MIDATL,LAG7_SOUTH,LAG7_WEST
0,0.000,0.000,0.000
1,0.000,0.000,0.000
2,0.000,0.000,0.000
3,0.000,0.000,0.000
4,0.000,0.000,0.000
...,...,...,...
43819,31223.944,11705.914,42521.702
43820,30934.643,11709.951,42381.205
43821,30619.720,11663.864,42197.353
43822,29955.427,11519.032,41556.164


In [8]:
# Setting the train-test split
ratio = 0.2
Num_test, Num_train = int(len(Load_DS) * ratio), len(Load_DS) - int(len(Load_DS) * ratio)

In [9]:
# Scaling the load data wrt WEST
mmScaler_load = MinMaxScaler()

# Splitting the data into train and test [LOAD]
Load_train, Load_test = Load_DS[:Num_train], Load_DS[Num_train:]
print("Raw load - Train: ")
display(Load_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 1]
Load_Lag1_train, Load_Lag1_test = Load_lag_1[:Num_train], Load_lag_1[Num_train:]
print("Raw lag 1 load - Train: ")
display(Load_Lag1_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 7]
Load_Lag7_train, Load_Lag7_test = Load_lag_7[:Num_train], Load_lag_7[Num_train:]
print("Raw lag 7 load - Train: ")
display(Load_Lag7_train.head(2))
print("\n")

Raw load - Train: 


,LOAD_MIDATL,LOAD_SOUTH,LOAD_WEST
0,31660.636,12571.309,49769.768
1,30766.066,12449.182,48714.914




Raw lag 1 load - Train: 


,LAG1_MIDATL,LAG1_SOUTH,LAG1_WEST
0,0.0,0.0,0.0
1,0.0,0.0,0.0




Raw lag 7 load - Train: 


,LAG7_MIDATL,LAG7_SOUTH,LAG7_WEST
0,0.0,0.0,0.0
1,0.0,0.0,0.0


In [10]:
# Scaling the data using mix-max scaler [TRAINING]
Load_train = mmScaler_load.fit_transform(Load_train)
print("Scaled load - Train: ")
display(Load_train)

Load_Lag1_train = mmScaler_load.transform(Load_Lag1_train)
print("Scaled lag 1 load - Train: ")
display(Load_Lag1_train)

Load_Lag7_train = mmScaler_load.transform(Load_Lag7_train)
print("Scaled lag 7 load - Train: ")
display(Load_Lag7_train)

Scaled load - Train: 


array([[0.43065024, 0.46215937, 0.50711927],
       [0.41002199, 0.45465502, 0.48722235],
       [0.39700564, 0.44521024, 0.47148018],
       ...,
       [0.50850536, 0.44495941, 0.46824365],
       [0.48777324, 0.42529905, 0.46160865],
       [0.45314908, 0.39972509, 0.42831099]])

Scaled lag 1 load - Train: 


array([[-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       ...,
       [ 0.45386016,  0.40100908,  0.44641797],
       [ 0.46159664,  0.41663176,  0.47836415],
       [ 0.45793298,  0.41360616,  0.47607833]])

Scaled lag 7 load - Train: 


array([[-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       ...,
       [ 0.47526171,  0.43326407,  0.44862886],
       [ 0.45846524,  0.4382983 ,  0.45863275],
       [ 0.43902549,  0.43183898,  0.44878566]])

In [11]:
# Scaling the data using mix-max scaler [TESTING]
Load_test = mmScaler_load.transform(Load_test)
print("Scaled load - Test: ")
display(Load_test)

Load_Lag1_test = mmScaler_load.transform(Load_Lag1_test)
print("Scaled lag 1 load - Test: ")
display(Load_Lag1_test)

Load_Lag7_test = mmScaler_load.transform(Load_Lag7_test)
print("Scaled lag 7 load - Test: ")
display(Load_Lag7_test)

Scaled load - Test: 


array([[0.420077  , 0.3703234 , 0.39565754],
       [0.38591386, 0.33897562, 0.36205661],
       [0.35196082, 0.30448827, 0.33430867],
       ...,
       [0.38274799, 0.37585456, 0.4469962 ],
       [0.35681528, 0.3556864 , 0.41965907],
       [0.33305307, 0.34057804, 0.3888862 ]])

Scaled lag 1 load - Test: 


array([[0.45516996, 0.40676053, 0.47418211],
       [0.43652896, 0.39468145, 0.45711798],
       [0.40442413, 0.37164187, 0.42807378],
       ...,
       [0.41847976, 0.31727448, 0.4592089 ],
       [0.37751442, 0.2897296 , 0.42478794],
       [0.34016058, 0.25654781, 0.38302336]])

Scaled lag 7 load - Test: 


array([[0.42808262, 0.43024529, 0.44452534],
       [0.41668037, 0.42899306, 0.43979978],
       [0.39615558, 0.41911268, 0.42448822],
       ...,
       [0.40664733, 0.40639953, 0.36428651],
       [0.39132913, 0.39750003, 0.35219224],
       [0.36766393, 0.3849365 , 0.33036574]])

In [12]:
## Transformation assigns perfectly.
Load_train[0]

array([0.43065024, 0.46215937, 0.50711927])

In [13]:
Load_Lag1_train[24]

array([0.43065024, 0.46215937, 0.50711927])

In [14]:
Load_Lag7_train[24*7]

array([0.43065024, 0.46215937, 0.50711927])

In [15]:
## Transformation assigns perfectly.
Load_test[5]

array([0.26187365, 0.22331367, 0.25204169])

In [16]:
Load_Lag1_test[24+5]

array([0.26187365, 0.22331367, 0.25204169])

In [17]:
Load_Lag7_test[24*7+5]

array([0.26187365, 0.22331367, 0.25204169])

## Processing the temperature data in train test split

In [18]:
# Fetching and concatenating the data
Temp_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Temperature"]) for fileName in fileList], axis= 1)
Temp_DS.columns = [i.removesuffix('.csv') for i in fileList]
Temp_DS = Temp_DS.add_prefix("TEMP_")

Temp_DS

,TEMP_MIDATL,TEMP_SOUTH,TEMP_WEST
0,-7.6,0.5,-9.0
1,-8.4,0.1,-9.0
2,-8.3,-0.5,-9.0
3,-6.8,-0.7,-9.0
4,-7.0,-0.7,-9.1
...,...,...,...
43819,4.8,9.9,-1.3
43820,4.4,10.2,-1.5
43821,3.7,9.6,-1.7
43822,2.8,9.0,-2.1


In [19]:
# Scaling the temperature data for each individual region
mmScaler_temp = MinMaxScaler()

# Splitting the data into train and test
Temp_train, Temp_test = Temp_DS[:Num_train], Temp_DS[Num_train:]
print("Raw temperature - Train: ")
display(Temp_train.head(5))
print("Raw temperature - Test: ")
display(Temp_test.head(5))
print("\n")

# Scaling the data using mix-max scaler
Temp_train = mmScaler_temp.fit_transform(Temp_train)
print("Scaled temperature - Train: ")
display(Temp_train)

Temp_test = mmScaler_temp.transform(Temp_test)
print("Scaled temperature - Test: ")
display(Temp_test)

Raw temperature - Train: 


,TEMP_MIDATL,TEMP_SOUTH,TEMP_WEST
0,-7.6,0.5,-9.0
1,-8.4,0.1,-9.0
2,-8.3,-0.5,-9.0
3,-6.8,-0.7,-9.0
4,-7.0,-0.7,-9.1


Raw temperature - Test: 


,TEMP_MIDATL,TEMP_SOUTH,TEMP_WEST
35060,5.1,5.2,9.8
35061,5.1,5.4,10.2
35062,5.3,5.6,11.1
35063,4.8,5.9,12.1
35064,4.3,7.2,12.9




Scaled temperature - Train: 


array([[0.29043478, 0.36007828, 0.29174664],
       [0.27652174, 0.35225049, 0.29174664],
       [0.27826087, 0.34050881, 0.29174664],
       ...,
       [0.52173913, 0.44031311, 0.60268714],
       [0.51652174, 0.43052838, 0.61612284],
       [0.51478261, 0.44618395, 0.63339731]])

Scaled temperature - Test: 


array([[0.51130435, 0.45205479, 0.65259117],
       [0.51130435, 0.45596869, 0.66026871],
       [0.51478261, 0.45988258, 0.67754319],
       ...,
       [0.48695652, 0.53816047, 0.4318618 ],
       [0.47130435, 0.52641879, 0.42418426],
       [0.46608696, 0.5146771 , 0.41458733]])

# Preparing the training and testing split for model input

In [20]:
X_train = np.array([[[i, j, k] for i,j,k in zip(Temp_train[m], Load_Lag1_train[m], Load_Lag7_train[m])] for m in range(len(Temp_train))])
print(X_train.shape)
print(X_train[0])

(35060, 3, 3)
[[ 0.29043478 -0.29942531 -0.29942531]
 [ 0.36007828 -0.31031077 -0.31031077]
 [ 0.29174664 -0.43165036 -0.43165036]]


In [21]:
X_test = np.array([[[i, j, k] for i,j,k in zip(Temp_test[m], Load_Lag1_test[m], Load_Lag7_test[m])] for m in range(len(Temp_test))])
print(X_test.shape)
print(X_test[0])

(8764, 3, 3)
[[0.51130435 0.45516996 0.42808262]
 [0.45205479 0.40676053 0.43024529]
 [0.65259117 0.47418211 0.44452534]]


In [22]:
# Setting up the batch and node parameters
num_batch, num_node = Load_DS.shape

number_feat = X_train.shape[2]

print("No. of batches: ", num_batch)
print("No. of nodes: " , num_node)
print("No. of features: ", number_feat)

No. of batches:  43824
No. of nodes:  3
No. of features:  3


In [23]:
edge_index = torch.LongTensor([np.repeat(range(num_node), num_node-1).tolist(),
                               [j for i in range(num_node) for j in range(num_node) if i != j]])

X_train, X_test = Variable(torch.FloatTensor(X_train)), torch.FloatTensor(X_test)
Y_train, Y_test = Variable(torch.FloatTensor(Load_train)), Load_test

In [24]:
edge_index

tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 0, 2, 0, 1]])

## DEFINING THE MODEL

In [25]:
torch.manual_seed(666)

model = Net(number_feat, edge_index, num_node)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=5e-4),
    dict(params=model.conv3.parameters(), weight_decay=0)
], lr=0.01) 

In [26]:
Epochs = 100

dataset = TensorDataset(X_train, Y_train)    
loader = DataLoader(dataset, batch_size = 64, shuffle=False)  

In [27]:
model.train()

for epoch in range(Epochs):
    loss = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()
        loss_train = 0
        for i in range(x.size(0)):
            output = model(x[i])
            loss_train += F.l1_loss(output[0][0], y[i,0]) + F.l1_loss(output[1][0], y[i,1]) + F.l1_loss(output[2][0], y[i,2])
        loss += loss_train/i
        loss_train.backward()
        optimizer.step()
    loss = loss/step
    print( "Epoch {}: the train loss = {:.4f}".format(epoch+1, loss))

Epoch 1: the train loss = 0.2723
Epoch 2: the train loss = 0.2082
Epoch 3: the train loss = 0.2038
Epoch 4: the train loss = 0.2007
Epoch 5: the train loss = 0.1997
Epoch 6: the train loss = 0.1997
Epoch 7: the train loss = 0.1972
Epoch 8: the train loss = 0.1982
Epoch 9: the train loss = 0.1961
Epoch 10: the train loss = 0.1941
Epoch 11: the train loss = 0.1937
Epoch 12: the train loss = 0.1932
Epoch 13: the train loss = 0.1924
Epoch 14: the train loss = 0.1934
Epoch 15: the train loss = 0.1929
Epoch 16: the train loss = 0.1916
Epoch 17: the train loss = 0.1929
Epoch 18: the train loss = 0.1924
Epoch 19: the train loss = 0.1923
Epoch 20: the train loss = 0.1916
Epoch 21: the train loss = 0.1914
Epoch 22: the train loss = 0.1916
Epoch 23: the train loss = 0.1923
Epoch 24: the train loss = 0.1910
Epoch 25: the train loss = 0.1914
Epoch 26: the train loss = 0.1916
Epoch 27: the train loss = 0.1899
Epoch 28: the train loss = 0.1859
Epoch 29: the train loss = 0.1866
Epoch 30: the train los

## MAKING PREDICTIONS

In [28]:
predictions = []

# Switching to eval mode
model.eval()

with torch.no_grad():
    for i in range(X_test.size(0)):
        p = model(X_test[i])
        predictions.append(torch.cat([p[0], p[1], p[2]], dim=-1).tolist())

predictions = np.array(predictions)

In [29]:
predictions

array([[0.37692305, 0.38987973, 0.43677685],
       [0.36797386, 0.37818122, 0.4279035 ],
       [0.35159346, 0.35676828, 0.41166189],
       ...,
       [0.34925845, 0.3537159 , 0.40934658],
       [0.33052489, 0.32922697, 0.39077175],
       [0.31482294, 0.30870095, 0.3752028 ]])

In [30]:
inversed_predictions = mmScaler_load.inverse_transform(predictions)
inversed_predictions

array([[29330.68954669, 11395.01808223, 46040.49740645],
       [28942.59675634, 11204.63449262, 45570.06863275],
       [28232.24032554, 10856.15672174, 44709.00417302],
       ...,
       [28130.97962279, 10806.48171625, 44586.25566425],
       [27318.5754086 , 10407.94481353, 43601.49311662],
       [26637.64080164, 10073.90088105, 42776.09080458]])

In [31]:
inversed_Y_test = mmScaler_load.inverse_transform(Y_test)
inversed_Y_test

array([[31202.114, 11076.755, 43860.518],
       [29720.587, 10566.596, 42079.133],
       [28248.171, 10005.343, 40608.049],
       ...,
       [29583.295, 11166.77 , 46582.286],
       [28458.691, 10838.55 , 45132.982],
       [27428.213, 10592.674, 43501.529]])

In [32]:
MAPE1 = mape(inversed_Y_test[:, 0].reshape(-1), inversed_predictions[:, 0].reshape(-1))
MAPE2 = mape(inversed_Y_test[:, 1].reshape(-1), inversed_predictions[:, 1].reshape(-1))
MAPE3 = mape(inversed_Y_test[:, 2].reshape(-1), inversed_predictions[:, 2].reshape(-1))

print( "The test mape is {:.3f}, {:.3f}, {:.3f}".format(MAPE1, MAPE2, MAPE3))

The test mape is 4.606, 4.661, 5.278


#### AVERAGE MAPE

In [33]:
meanMAPE = np.mean([MAPE1, MAPE2, MAPE3])
meanMAPE

4.848477837739716

#### WEIGHTED MAPE

In [34]:
# Scaled MAPE with max values
weightMax = pd.Series(Load_DS.max() / sum(Load_DS.max())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the max load of a region: ", weightedMAPE)

0    0.366928
1    0.138851
2    0.494221
dtype: float64


0    4.606199
1    4.661074
2    5.278161
dtype: float64


Weighted using the max load of a region:  4.945915973851104


In [35]:
# Scaled MAPE with average values
weightMax = pd.Series(Load_DS.mean() / sum(Load_DS.mean())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the mean load of a region: ", weightedMAPE)

0    0.345855
1    0.125173
2    0.528972
dtype: float64


0    4.606199
1    4.661074
2    5.278161
dtype: float64


Weighted using the mean load of a region:  4.968516614565884
